In [1]:
from scipy.stats import qmc
import numpy as np

import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
import shutil
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                                                                                                                                                                       
import tempfile 
import random
import re

import modp as mp

In [2]:
random.seed(32)

#### Read in min and max values for each parameter and pft 

In [3]:
param_ranges_full = pd.read_csv('/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/sr_ensemble_params.csv')
param_ranges = param_ranges_full[['param', 'value_mean', 'value_min', 'value_max', 'pft', 'organ']]

# number of parameters
n_params = len(param_ranges)

# number of PFTs - some are global so subtract one
n_pfts = len(pd.unique(param_ranges['pft'])) - 1

param_names = list(param_ranges.param)
pfts = list(param_ranges.pft)
organs = list(param_ranges.organ)

print(param_ranges)

                                  param  value_mean  value_min  value_max  \
0           fates_alloc_storage_cushion        1.20      1.000      1.800   
1       fates_alloc_store_priority_frac        0.80      0.640      0.960   
2                     fates_allom_d2bl1        0.05      0.025      0.075   
3                     fates_allom_d2bl2        1.50      1.200      1.800   
4               fates_allom_fnrt_prof_a        7.00      5.600      8.400   
..                                  ...         ...        ...        ...   
72  fates_recruit_seed_germination_rate        0.50      0.250      0.750   
73                   fates_wood_density        0.53      0.265      0.795   
74                  fates_turnover_leaf        1.50      0.750      2.250   
75                fates_turnover_branch       80.00     40.000    120.000   
76                  fates_turnover_fnrt        1.50      0.750      2.250   

    pft  organ  
0     2      0  
1     2      0  
2     2      0  
3     2

In [6]:
np.shape(scaled_sample)

(1600, 77)

In [4]:
n_inst = 1600

sampler = qmc.LatinHypercube(d=n_params)
sample = sampler.random(n=n_inst)

# scale to parameter ranges
l_bounds = param_ranges['value_min']
u_bounds = param_ranges['value_max']

scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

print(scaled_sample[0,:])

[ 1.74348200e+00  8.45039943e-01  6.83328170e-02  1.69611383e+00
  7.97497434e+00  1.97888214e+00  1.07221679e+00  1.09479456e+04
  1.54564304e-01  1.08399467e-02  9.28573496e+00  8.74258662e+01
  3.15029427e-06  2.02227734e+00 -7.04942882e+04 -1.70032856e+05
  4.65201687e-01  2.42170908e-01  4.37756357e-01  2.92304656e-01
  4.24635809e-01  1.78839443e-02 -2.31076851e+01  3.30695728e-01
  2.26810025e+00  4.45040806e-01  6.75137633e-01  1.33596388e-06
  4.40668907e-01  1.53487055e+00  2.43260649e-01  1.44022179e-01
  7.19166152e+01  3.47664979e-01  3.47416787e-01  5.11042529e+00
  7.11991407e+01  1.94433249e+00  8.14613810e-01  7.69380379e-01
  1.40434168e+00  6.80219034e-01  7.23097414e-02  1.65523407e+00
  7.59638543e+00  1.19138774e+00  1.07007646e+00  1.39159282e+04
  1.35902490e-01  7.51433442e-03  6.37263733e+00  7.59806341e+01
  2.43088234e+00 -5.77781513e+04 -2.83220065e+05  4.85961838e-01
  2.71117123e-01  5.44001915e-01  2.98871060e-01  5.26456286e-01
  1.06975101e-02 -3.73480

In [5]:
## Read in defaut FATES file - note that this is the default for FATES but with:
# - updated allometries for tropical PFTs
# - size bins that are consistent with the DBEN protocol. 
# - supplemental seed rain
# - updated vai bins
# - atkin respiration 
# - size dependent mortality 

input_fname = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/fates_params_sr_base.nc'

# for each sample 
for i in range(0,n_inst) :
    
    ens_num = str(i+1).zfill(4)
    # final parameter file name
    fout = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/fates_params_sr_ens_'+ens_num+'.nc'
    
    shutil.copyfile(input_fname, fout)                                                                                                                             
   
    # loop through each parameter and apply either to the correct pft or globally
    for j in range(0, n_params) : 
        
        var = param_names[j]
        pft = pfts[j]
        organ = organs[j]
        
        val = scaled_sample[i, j]
        
        mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
        
#         if var == 'fates_mort_bmort' and pft == 1 : 
#             pft = pft + 1
#             val = val * 0.75
#             mp.main(var = var, pft = pft, fin = fout, val = val, 
#                     fout = fout, O = 1, organ = organ)
            
#         if var == 'fates_wood_density' and pft == 1 : 
#             pft = pft + 1
#             val = val * 1.5
#             mp.main(var = var, pft = pft, fin = fout, val = val, 
#                     fout = fout, O = 1, organ = organ)
            
#         if var == 'fates_leaf_vcmax25top' and pft == 1 : 
#             pft = pft + 1
#             val = val * 0.9
#             mp.main(var = var, pft = pft, fin = fout, val = val, 
#                     fout = fout, O = 1, organ = organ)

    # parameters constrained by the sum equals to one
    idx_frflig = [k for k, x in enumerate(param_names) if x == 'fates_frag_fnrt_flig']
    idx_frfcel = [k for k, x in enumerate(param_names) if x == 'fates_frag_fnrt_fcel']
    idx_lfflig = [k for k, x in enumerate(param_names) if x == 'fates_frag_leaf_flig']
    idx_lffcel = [k for k, x in enumerate(param_names) if x == 'fates_frag_leaf_fcel']
    idx_cwfcel = param_names.index('fates_frag_cwd_fcel')
    
    val_frflab = 1.0 - scaled_sample[i, idx_frflig[0]] - scaled_sample[i, idx_frfcel[0]]
    mp.main(var = 'fates_frag_fnrt_flab', pft = pfts[idx_frflig[0]], fin = fout, val = val_frflab, 
                    fout = fout, O = 1, organ = organs[idx_frflig[0]])

    val_frflab = 1.0 - scaled_sample[i, idx_frflig[1]] - scaled_sample[i, idx_frfcel[1]]
    mp.main(var = 'fates_frag_fnrt_flab', pft = pfts[idx_frflig[1]], fin = fout, val = val_frflab, 
                    fout = fout, O = 1, organ = organs[idx_frflig[1]])
    
    val_lfflab = 1.0 - scaled_sample[i, idx_lfflig[0]] - scaled_sample[i, idx_lffcel[0]]
    mp.main(var = 'fates_frag_leaf_flab', pft = pfts[idx_lfflig[0]], fin = fout, val = val_lfflab, 
                    fout = fout, O = 1, organ = organs[idx_lfflig[0]])

    val_lfflab = 1.0 - scaled_sample[i, idx_lfflig[1]] - scaled_sample[i, idx_lffcel[1]]
    mp.main(var = 'fates_frag_leaf_flab', pft = pfts[idx_lfflig[1]], fin = fout, val = val_lfflab, 
                    fout = fout, O = 1, organ = organs[idx_lfflig[1]])
    
    val_cwflig = 1.0 - scaled_sample[i, idx_cwfcel]
    mp.main(var = 'fates_frag_cwd_flig', pft = 0, fin = fout, val = val_cwflig, 
                    fout = fout, O = 1, organ = 0)
    

In [2]:
## in defaut FATES file - note that this is the default for FATES but with:
# - updated allometries for tropical PFTs
# - size bins that are consistent with the DBEN protocol. 
# - supplemental seed rain
# - updated vai bins
# - atkin respiration 
# - size dependent mortality 

list_can = np.array([  18,  135,  267,  291,  331,  352,  386,  420,  463,  471,  519,
        541,  567,  834,  883,  916,  938,  994, 1006, 1068, 1241, 1246,
       1374, 1431, 1500, 1530, 1580])-1

# for each sample 
for i in list_can:
    
    ens_num = str(i+1).zfill(4)
    # final parameter file name
    fout = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/candidates/fates_params_sr_ens_'+ens_num+'.nc'
    
   
    # loop through each parameter and apply either to the correct pft or globally
    mp.main(var = 'fates_landuse_logging_collateral_frac', fin = fout, val = 0.02, 
                    fout = fout, O = 1, pft=0, organ=0)
    mp.main(var = 'fates_landuse_logging_direct_frac', fin = fout, val = 0.95, 
                    fout = fout, O = 1, pft=0, organ=0)
    mp.main(var = 'fates_landuse_logging_mechanical_frac', fin = fout, val = 0.03, 
                    fout = fout, O = 1, pft=0, organ=0)
    mp.main(var = 'fates_landuse_logging_dbhmin', fin = fout, val = 30, 
                    fout = fout, O = 1, pft=0, organ=0)
 

In [24]:
tmp = scaled_sample[0:50,:]

df  = pd.DataFrame(data=tmp, columns = param_names)

In [25]:
df

,fates_alloc_storage_cushion,fates_alloc_store_priority_frac,fates_allom_d2bl1,fates_allom_d2bl2,fates_allom_fnrt_prof_a,fates_allom_fnrt_prof_b,fates_allom_l2fr,fates_leaf_stomatal_intercept,fates_grperc,fates_leaf_slatop,...,fates_mort_scalar_cstarvation,fates_recruit_height_min,fates_recruit_init_density,fates_recruit_seed_alloc,fates_recruit_seed_dbh_repro_threshold,fates_recruit_seed_germination_rate,fates_wood_density,fates_turnover_leaf,fates_turnover_branch,fates_turnover_fnrt
0,1.327828,0.587413,0.058974,1.303389,6.639592,1.925530,0.828076,7480.102828,0.124694,0.004385,...,0.391330,1.762789,0.233315,0.087958,57.155309,0.624889,0.603365,1.542131,42.597856,2.010295
1,1.791669,0.667451,0.073401,1.219912,6.854915,1.797244,1.489892,9617.189968,0.058906,0.003437,...,0.472496,1.531372,0.185871,0.109482,55.909417,0.348684,0.443164,2.025702,85.928317,1.939489
2,1.784124,0.944891,0.037141,1.396908,8.064929,2.195765,0.711057,9160.203513,0.159910,0.003887,...,0.633899,1.012000,0.174508,0.137546,52.627902,0.422912,0.567694,1.418520,103.785791,2.246691
3,1.631287,1.072249,0.069268,1.578768,5.817032,1.830828,0.628715,5598.060552,0.100820,0.007392,...,0.894106,1.548778,0.224697,0.058520,77.353749,0.272179,0.693767,0.981646,100.875787,0.793298
4,1.263745,0.908010,0.036085,1.399266,6.026211,1.802299,0.977823,9655.298430,0.109197,0.005914,...,0.568478,0.956262,0.242406,0.145981,80.191215,0.390746,0.425711,1.325525,111.354124,1.695713
5,1.652301,0.607769,0.062518,1.404926,7.385890,2.320260,1.411550,7205.790185,0.070157,0.005289,...,0.344219,1.727787,0.292392,0.127753,109.570395,0.251184,0.665990,1.197785,81.277532,1.870776
6,1.546898,0.871548,0.065165,1.648011,5.897651,1.755866,0.817685,10360.336749,0.164469,0.003004,...,0.889075,0.793674,0.168170,0.133185,44.620917,0.363779,0.792957,1.352162,79.430997,1.869173
7,1.163935,0.456995,0.034925,1.406892,5.750955,1.990725,1.463812,13870.952262,0.136041,0.003360,...,0.532103,1.948313,0.275423,0.058105,61.303371,0.664696,0.748968,1.859697,94.444206,2.064177
8,1.360810,0.804078,0.058665,1.521229,7.403444,2.213897,0.649023,12996.283326,0.072364,0.004001,...,0.346011,1.262845,0.263411,0.056965,40.494546,0.717999,0.282274,2.031495,116.401552,1.459219
9,1.104800,0.601399,0.025545,1.630789,7.227837,1.837581,0.880183,13710.404638,0.144280,0.003534,...,0.500055,0.687450,0.147520,0.078010,43.120682,0.373162,0.394101,1.358593,51.449505,0.875961
